In [1]:
!pip install autograd
from autograd import grad

  Created wheel for autograd: filename=autograd-1.3-py3-none-any.whl size=47988 sha256=315471e8277bb7dbae65c77823a679d53e0d1d0801c1510bab4035a2bbf5dc41
  Stored in directory: c:\users\malko\appdata\local\pip\cache\wheels\85\f5\d2\3ef47d3a836b17620bf41647222825b065245862d12aa62885
Successfully built autograd


In [2]:
def find_root(f, l, r, eps=1e-6, method='newton'):
  if method == 'bisection':
    while r-l > eps:
      m = l+(r-l)/2
      x, y, z = f(l), f(r), f(m)
      if x <= 0 and y >= 0:
        if z <= 0:
          l = m
        else:
          r = m
      elif x >= 0 and y <= 0:
        if z <= 0:
          r = m
        else:
          l = m
      else:
        assert False
    return l
  elif method == 'newton':
    f_ = grad(f)
    xs = [l+(r-l)/2]
    while True:
      x = xs[-1]
      xs.append(-f(x)/f_(x)+x)
      if abs(xs[-1]-x) < eps:
        return xs[-1]

In [3]:
3**(1/3)-find_root(lambda x: x**3-3, 0, 2)

-3.3306690738754696e-14

In [4]:
3**(1/3)-find_root(lambda x: x**3-3, 0, 2, method='bisection')

2.722117051767725e-07

In [24]:
from collections import *

# malkovsky: идея норм, но этот код даже менее эффективен тому, чтобы просто циклом от l до r пройти с шагом eps,
# так как реальности вы делите на отрезки длины (r - l) / 2^k с таким k, что величина попадает в отрезок [eps/2, eps]
# если бы какой-то break был, то норм, а так просто излишне усложненный код.

def find_roots(f, l, r, calls, eps=1e-6, method='newton'):
  calls[0] += 1
  x, y = f(l), f(r)
  if r-l < eps:
    if x*y <= 0:
      return deque([find_root(f, l, r, method=method)])
    return deque()
    
  m = l+(r-l)/2
  a = find_roots(f, l, m, calls, eps, method=method) # malkovsky: missed method=method
  b = find_roots(f, m, r, calls, eps, method=method) # malkovsky: missed method=method
  
  c = deque()
  while a or b:
    print(a, b)
    if a:
      if not b:
        c.append(a.popleft())
      else:
        if a[0] < b[0]:
          c.append(a.popleft())
        else:
          c.append(b.popleft())
    else:
      c.append(b.popleft())
    if len(c) > 1 and c[-1]-c[-2] < eps:
        c.pop()
  
  return c

In [25]:
calls = [0]
find_roots(lambda x: x*(x-1)*(x-2)*(x-3), -10., 10., calls)
print(f"Recurrent calls: {calls}")

deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.628325852704105e-13])
deque([]) deque([-1.62832585270410

In [7]:
def find_minimum(f, l, r, max_extrema, f_, eps=1e-6, method='newton'):
  if max_extrema > 1:
    xs = find_roots(f_, l, r, max_extrema, method=method)
    return min(xs, key=f)
    
  while r-l > eps:
    m1 = l+(r-l)/3
    m2 = l+2*(r-l)/3
    if f(m1) < f(m2):
      r = m2
    else:
      l = m1
      
  return l

In [8]:
find_minimum(f := lambda x: x*(x-1)*(x-2)*(x-3), -10., 10., 3, grad(f))

2.618033988749895

In [9]:
find_minimum (f := lambda x: x**2, -2, 3, 1, None)

-4.069697706341903e-07

In [10]:
a = 1
b = 2
c = 3
d = 4

import autograd.numpy as np
from autograd import grad
f = lambda x: np.exp(a*x)+np.exp(-b*x)+c*(x-d)**2
f_ = lambda x: np.exp(a*x)*a+np.exp(-b*x)*(-b)+2*c*(x-d)

In [11]:
# malkovsky: границы должны были подбираться из аргументов
find_minimum(f, -1000., 1001., 1, f_)

2.315283478948846